# Get the PanSTARRS DR1 photometry for the CGM$^2$ Galaxy Catalog

In [42]:
from astroquery.vizier import Vizier
import astropy.units as u
import astropy.coordinates as coord
from astropy.table import Table, vstack

In [5]:
def panstarrs_query(ra_deg, dec_deg, rad_deg, maxmag=24,
                    maxsources=10000):
    """
    Query PanSTARRS @ VizieR using astroquery.vizier
    
    (from https://michaelmommert.wordpress.com/2017/02/13/
    accessing-the-gaia-and-pan-starrs-catalogs-using-python/)
    
    :param ra_deg: RA in degrees
    :param dec_deg: Declination in degrees
    :param rad_deg: field radius in degrees
    :param maxmag: upper limit G magnitude (optional)
    :param maxsources: maximum number of sources
    :return: astropy.table object
    """
    vquery = Vizier(columns=['objID', 'RAJ2000', 'DEJ2000',
                             'e_RAJ2000', 'e_DEJ2000',
                             'gmag', 'e_gmag',
                             'rmag', 'e_rmag',
                             'imag', 'e_imag',
                             'zmag', 'e_zmag',
                             'ymag', 'e_ymag'],
                    column_filters={"gmag":
                                    ("<%f" % maxmag)},
                    row_limit=maxsources)

    field = coord.SkyCoord(ra=ra_deg, dec=dec_deg,
                           unit=(u.deg, u.deg),
                           frame='icrs')
    return vquery.query_region(field,
                               width=("%fd" % rad_deg),
                               catalog="II/349/ps1")[0]


# Example query
print(panstarrs_query(12.345, 67.89, 0.01))

      objID           RAJ2000       DEJ2000    ...  e_zmag   ymag   e_ymag
                        deg           deg      ...   mag     mag     mag  
------------------ ------------- ------------- ... ------- ------- -------
189460123532453147  12.353199480  67.885283260 ...  0.0104 16.9197  0.0101
189470123537790102  12.353740850  67.891079330 ...  0.0050 16.2947  0.0062
189470123534421848  12.353373340  67.892536640 ...  0.0156 18.3633  0.0304
189460123528446259  12.352828940  67.887895580 ...  0.0232 17.9945  0.0176
189460123423483398  12.342315510  67.885502260 ...  0.0031 16.6284  0.0054


### Read in CGM$^2$ ra decs

RA is in hourangle. Convert to degrees like a sane person

In [21]:
cgmsq_radec_table = Table.read('cgmsquared_gal_radec.txt', format='ascii')
cgmsq_radec_table

ID,RA,DEC
int64,float64,float64
146,9.591402,2.08156
159,9.591022,2.06473
293,9.59142,2.10502
343,9.590647,2.04932
393,9.591168,2.10232
432,9.59091,2.09427
456,9.590219,2.04628
480,9.5904,2.07028
497,9.590591,2.09556


In [22]:
cgmsq_radec_table['RA'] = (cgmsq_radec_table['RA']*u.hourangle).to(u.degree)
cgmsq_radec_table['DEC'] = cgmsq_radec_table['DEC']*u.degree
ra = cgmsq_radec_table['RA']
dec = cgmsq_radec_table['DEC']

In [23]:
cgmsq_radec_table

ID,RA,DEC
,deg,deg
int64,float64,float64
146,143.87103,2.08156
159,143.86533,2.06473
293,143.87129999999996,2.10502
343,143.859705,2.04932
393,143.86751999999998,2.10232
432,143.86364999999998,2.09427
456,143.85328499999997,2.04628
480,143.856,2.07028


## Query a few of the galaxies

* We don't have a one-to-one matching. Not unexpected
* Are there more than one match at each ra,dec?

In [29]:
panstarrs_query(ra[:5], dec[:5], 0.001)

_q,objID,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,ymag,e_ymag
,,deg,deg,arcsec,arcsec,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag
int32,int64,float64,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32
1,110491438706598496,143.870658980,2.081683290,0.0043,0.0080,18.3108,0.0058,17.9118,0.0072,17.7221,0.0025,17.6875,0.0028,17.6459,0.0202
2,110471438650128304,143.865012120,2.064859860,0.0094,0.0049,18.1937,0.0034,17.7480,0.0045,17.5897,0.0080,17.5213,0.0040,17.4816,0.0100


## Slice up the query so it doesnt time out and query the whole cgmsquared catalog

In [44]:
radius = 0.001 # degrees ~ 3.6 arcsec
panstarrs_query_result = [panstarrs_query(ra[:500], dec[:500], radius),
                          panstarrs_query(ra[500:1000], dec[500:1000], radius),
                          panstarrs_query(ra[1000:1500], dec[1000:1500], radius),
                          panstarrs_query(ra[1000:1500], dec[1000:1500], radius),
                          panstarrs_query(ra[1500:2000], dec[1500:2000], radius),
                          panstarrs_query(ra[2000:2500], dec[2000:2500], radius),
                          panstarrs_query(ra[2500:], dec[2500:], radius)
                         ]

### Stack the queries into a table

In [45]:
panstarrs_querry_table = vstack(panstarrs_query_result)

In [46]:
panstarrs_querry_table

_q,objID,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,ymag,e_ymag
,,deg,deg,arcsec,arcsec,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag
int32,int64,float64,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32
1,110491438706598496,143.870658980,2.081683290,0.0043,0.0080,18.3108,0.0058,17.9118,0.0072,17.7221,0.0025,17.6875,0.0028,17.6459,0.0202
2,110471438650128304,143.865012120,2.064859860,0.0094,0.0049,18.1937,0.0034,17.7480,0.0045,17.5897,0.0080,17.5213,0.0040,17.4816,0.0100
29,110461438137211322,143.813725910,2.050789160,0.0188,0.0253,21.0842,0.1254,20.6095,0.0621,20.3790,0.0402,20.2169,0.0400,19.8899,0.1770
30,110481438158739144,143.815900110,2.073902600,0.0128,0.0147,21.6445,0.0910,20.2694,0.0571,19.7759,0.0482,19.4905,0.0292,19.0490,0.1271
31,110531438182686912,143.818258320,2.113701090,0.0084,0.0059,19.8143,0.0083,18.5967,0.0057,17.2301,0.0045,16.6325,0.0027,16.3242,0.0016
32,110461438085871455,143.808371710,2.050833570,0.0256,0.0298,22.0396,0.1574,21.4074,0.0661,21.0586,0.0540,20.7444,0.0474,--,--
33,110501438107128189,143.810694090,2.089812340,0.1189,0.1211,22.0913,0.1841,21.4764,0.0146,21.2562,0.0642,--,--,--,--
33,110501438109818345,143.810937930,2.089879720,0.0690,0.0795,21.8527,0.1881,21.5313,0.0111,21.3107,0.0832,--,--,--,--


# Read in the DESI matches

In [47]:
desi_cat = Table.read('/Users/mwilde/Dropbox/COS-Gemini/Imaging/DESI/crossmatch_decals_cgmsq_trimmed.fits')

In [50]:
desi_cat[desi_cat['dered_mag_r'] > 23]

/anaconda3/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


ra_cs,dec_cs,ra_decals,dec_decals,mag_g,mag_r,mag_z,mag_w1,mag_w2,mag_w3,mag_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,type,dist_arcsec
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,float64
143.871302604675,2.10502004623413,143.870885070929,2.1051104267656697,24.6122,23.339000000000002,22.6102,22.5812,nan,18.5455,14.9747,24.4831,23.2521,22.5615,22.5738,nan,18.5445,14.9743,PSF,1.5369422039130198
143.855996131897,2.07028007507324,143.85574813479,2.07039896564623,25.8625,23.677,21.7373,20.8935,21.1883,18.6238,16.1402,25.7286,23.5868,21.6869,20.8858,21.1836,18.6228,16.1398,PSF,0.98955654628718
143.858871459961,2.09556007385254,143.858539348648,2.0956844568568798,25.6851,23.3867,22.1129,22.063000000000002,nan,nan,nan,25.5534,23.2979,22.0632,22.0554,nan,nan,nan,REX,1.27595279486804
143.848886489868,2.0627601146698002,143.848620843855,2.06288255336767,24.6619,23.6039,22.0399,20.2599,20.4737,18.6678,14.6915,24.5267,23.5128,21.989,20.2522,20.4689,18.6668,14.6911,PSF,1.05245436419876
143.851761817932,2.08786010742188,143.851461335554,2.0880072657577897,25.3585,23.5076,21.3194,19.2714,19.8052,nan,15.8515,25.2251,23.4177,21.2691,19.2637,19.8005,nan,15.8511,REX,1.20385090795178
143.82398128509502,2.07069993019104,143.823718338261,2.0708098636631496,25.3426,23.1946,21.8913,nan,nan,nan,15.3989,25.2052,23.101999999999997,21.8395,nan,nan,nan,15.3985,REX,1.02543857803317
143.811163902283,2.10125994682312,143.810869905213,2.10132525893738,24.3582,23.56,22.6727,21.7166,nan,18.2705,16.4814,24.2217,23.4681,22.6213,21.7087,nan,18.2694,16.480999999999998,EXP,1.08349681405896
143.790693283081,2.11844992637634,143.79055302059402,2.11841214175159,24.3069,23.4461,22.9704,nan,nan,18.6351,16.4426,24.1697,23.3537,22.9187,nan,nan,18.634,16.4422,REX,0.522612393079936
143.786129951477,2.0355799198150604,143.786108544021,2.0356299471549,25.339000000000002,23.7266,22.2417,21.3571,21.633000000000003,nan,16.0995,25.1999,23.6328,22.1893,21.3491,21.6281,nan,16.0991,REX,0.195875589527132


In [53]:
ra_desi = desi_cat[desi_cat['dered_mag_r'] > 23]['ra_decals'][0]
dec_desi = desi_cat[desi_cat['dered_mag_r'] > 23]['dec_decals'][0]

/anaconda3/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


In [52]:
panstarrs_querry_table[(panstarrs_querry_table['RAJ2000']]

3.6